In [8]:
# select ASm6As that have the same trend across samples which contains the same ASm6A
from collections import Counter

#### ASm6A
asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
###########
# Prop = 0.8
###########

def select_functional_ASm6A(prop):
    result_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/optimal_cutoff/"
    result_file = os.path.join(result_dir, "ASm6A_%s.bed" % str(prop))
    # os.chdir(asm6a_dir)
    # result_dir = "common_ASm6A/totaltissues/"
    # os.system("mkdir -p %s" % result_dir)
    # result_file = os.path.join(result_dir, "common_ASm6A.bed")  #########
    #####################################################################################
    m6a_dict = {}
    bed_list = glob.glob("%s/*.bed" % asm6a_dir)
    for bed in bed_list:
        if os.path.getsize(bed):
            sample = os.path.basename(bed)
            df = pd.read_table(bed, header=None)
            ## chr1:846078:ref>alt
            df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str) + ":" + df.iloc[:,3].str.split(";").str[0]
            df['mark'] = df.iloc[:,3].str.split(";").str[-1]
            term_list, mark_list = df['term'].tolist(), df['mark'].tolist()
            for i in range(len(term_list)):
                m6a_dict[term_list[i]] = m6a_dict.get(term_list[i], []) + [mark_list[i]]

    final_list = []
    for term, mark_list in m6a_dict.items():
        ## appear at least twice
        if len(mark_list) >= 2: 
            ## the mark must be same across all samples
            count, new_line = Counter(mark_list), ""
            if (count['ref']/(count['ref']+count['alt'])) >= prop:
                new_line = [term.split(":")[0], int(term.split(":")[1])-1, int(term.split(":")[1]), "ref", term.split(":")[2]]
                final_list.append(new_line)
            elif (count['alt']/(count['ref']+count['alt'])) >= prop:
                new_line = [term.split(":")[0], int(term.split(":")[1])-1, int(term.split(":")[1]), "alt", term.split(":")[2]]
                final_list.append(new_line)
    df = pd.DataFrame(final_list)
    print(prop, len(df))
    df.drop_duplicates().to_csv(result_file, sep="\t", header=False, index=False)

    
##### run
for prop in range(10, 105, 5):
    prop = prop / 100
    select_functional_ASm6A(prop)

0.1 6843
0.15 6843
0.2 6843
0.25 6843
0.3 6843
0.35 6843
0.4 6843
0.45 6843
0.5 6843
0.55 5395
0.6 5111
0.65 4601
0.7 3490
0.75 3297
0.8 2801
0.85 2433
0.9 2285
0.95 2244
1.0 2243


In [1]:
#### split into different tissues
os.chdir("/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/")
result_dir = "by_sample"
os.system("mkdir -p %s" % result_dir)
##
ASm6A_dir = "/Charles/project/ASm6A/ASm6A/"
bed_list = glob.glob("%s/*.bed" % ASm6A_dir)
####
functional_ASm6A = "common_ASm6A.bed"

for asm6A in bed_list:
    res = os.path.join(result_dir, os.path.basename(asm6A))
    os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" % (asm6A, functional_ASm6A, res))